<center style = "font-family: 'Lucida Console', 'Courier New', monospace;">
    <img src = "https://www.bargainmarket.com.au/images/banner-pets-dog-cat-boarding.png" width=600 height = 400>
    <h1 style = "background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%);border-radius: 20px; font-size:30px">PetFinder.my - Pawpularity Contest 🐶🐈</h1>
</center>

<div style = "background: rgb(224,224,224);border-radius: 42px;">
    <h1 style = "font-family: Consolas; text-align:center; color:#FF69B4">Sponsors 💰</h1>
    <h2 style = "font-family: Consolas; text-align:center">PetFinder.my</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    PetFinder.my is Malaysia’s leading animal welfare platform, featuring over 180,000 animals with 54,000 happily adopted. PetFinder collaborates closely with animal lovers, media, corporations, and global organizations to improve animal welfare.
        <a href = "https://petfinder.my/">Check their official website</a>
</div>

<div style = "background: rgb(224,224,224);border-radius: 42px;">
    <h1 style = "font-family: Consolas; text-align:center; color:#FF69B4">Introduction</h1>
    <h2 style = "font-family: Consolas; text-align:center">Why this Competition ❓</h2>
    <p style = "font-family : Lucida Sans Typewriter">
        Currently, PetFinder.my uses a basic <a href = "https://petfinder.my/cutenessmeter">Cuteness Meter</a> to rank pet photos. It analyzes picture composition and other factors compared to the performance of thousands of pet profiles. While this basic tool is helpful, it's still in an experimental stage and the algorithm could be improved.
    </p>
    <h2 style = "font-family: Consolas; text-align:center">Goal of Competition 🥅</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    The goal of this competition is to predict "PawPularity" of pet using raw images and metadata which will guide shelters and rescuers around the world to improve the appeal of their pet profiles, automatically enhancing photo quality and recommending composition improvements. As a result, stray dogs and cats can find their "furever" homes much faster.
    </p>
</div>

<h2 style = "font-family: Consolas">More Details</h2>
<p style = "font-family : Lucida Sans Typewriter">Check <a href = "https://www.kaggle.com/c/petfinder-pawpularity-score/data">competition page</a> for details</p>
<h2 style = "font-family : Comic Sans MS">Let's dive in ⬇️</h2>

<center><img src = "https://img.shields.io/badge/Upvote-If%20you%20found%20this%20notebook%20useful-blue" width=400 height = 400></center>

<p style = "font-family : Lucida Sans Typewriter">Thanks @debarshichanda for wonderful <a href = "https://www.kaggle.com/debarshichanda/pytorch-w-b-pawpularity-training">notebook</a>. It helped me to get started with PyTorch</p>

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Install Dependencies</h1>

In [ ]:
!pip install --upgrade wandb
!pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install git+https://github.com/rwightman/pytorch-image-models

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Import Libraries</h1>

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
import copy 
import time

import wandb

import numpy as np
from collections import defaultdict
import gc

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error as mse

import pandas as pd
from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import seaborn as sns
sns.set()

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torch.optim as optim
from torch.optim import lr_scheduler as LR
import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2

from colorama import Fore, Back, Style

from IPython.display import IFrame
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Initializing Constants</h1>

In [ ]:
TRAIN_CONFIG = dict(
    height = 128,
    width = 128,
    channels = 3,
    num_classes = 1,
    n_splits = 5,
    batch_size = 32,
    model = "tf_efficientnet_b4_ns",
    pretrained = True,
    epochs = 10,
    seed = 2021,
    learning_rate = 0.0001,
    min_lr = 1e-6,
    T_max = 100,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
)

In [ ]:
b_crma = Fore.BLUE
r_crma = Fore.RED
y_crma = Fore.YELLOW
sr_crma = Style.RESET_ALL

TRAIN_PATH = "../input/petfinder-pawpularity-score/train"
CSV_PATH = "../input/petfinder-pawpularity-score/train.csv"

train_df = pd.read_csv(CSV_PATH)
feature_cols = list(train_df.columns)[1:-1]
train_df["Filepath"] = train_df["Id"].apply(lambda x : os.path.join(TRAIN_PATH,x+".jpg"))

bin_ranges = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
bin_names = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

pseudo_labels = np.array(pd.cut(np.array(train_df["Pawpularity"]),bins=bin_ranges,labels=bin_names))
files_ls = list(train_df["Filepath"])

rand_img = torch.randn(1,3,TRAIN_CONFIG["height"],TRAIN_CONFIG["width"]).to(TRAIN_CONFIG["device"])
rand_meta = torch.randn(1,len(feature_cols)).to(TRAIN_CONFIG["device"])
    
print(f"Number of Training Images : {len(train_df)} \n\n")
train_df.head()

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Data Visualization 📊💹</h1>

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Weights and Biases</h2>
    <center><img src = "https://i.imgur.com/KISYcqD.png" width=200 height = 200></center>
    <a href = ""; style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; font-size:25px">Checkout Dashboard created for this notebook</a>
    <p style = "font-family : Lucida Sans Typewriter">
    Weights and Biases is a set of Machine Learning tools used for experiment tracking, dataset versioning, and collaborating on ML projects. Weights and Biases is useful in many applications such as
    </p>  
    <ul>
          <li>Experiment Tracking</li>
          <li>Hyperparameter Tuning</li>
          <li>Data Visualization</li>
          <li>Data and model Versioning</li>
          <li>Collaborative Reports</li>
    </ul>
    <a href = "https://wandb.ai/site">Go to offocial website for more tutorials and Documentation</a>
</div>

In [ ]:
# Check https://www.kaggle.com/debarshichanda/pytorch-w-b-pawpularity-training for more details
try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('''If you want to use your W&B account, Follow these steps :
            -> go to Add-ons {Below name of notebook} -> Secrets -> Add a new Secret
            -> Label = wandb_api
            -> Value = W&B access token from https://wandb.ai/authorize 
         ''')

In [ ]:
WANDB_CONFIG = {
   "project_name" : "PetFinder.my - Pawpularity Contest",
   "group_name" : "PyTorch Training",
   "job_type_data" : "Data Visualization",
    "job_type_train" : "Training",
   "anonymity" : "must",
    "artifact" : "training_data"
}

run = wandb.init(
    project = WANDB_CONFIG["project_name"],
    group = WANDB_CONFIG["group_name"],
    job_type = WANDB_CONFIG["job_type_data"],
    anonymous= WANDB_CONFIG["anonymity"]
)

wb_table = wandb.Table(columns = [
    'Id', 'Image', 'Pawpularity', 'Subject Focus', 'Eyes', 'Face',
    'Near', 'Action', 'Accessory', 'Group', 'Collage', 
     'Human', 'Occlusion', 'Info', 'Blur'
])

In [ ]:
for i in tqdm(range(len(train_df))):
    row = train_df.loc[i]
    impath = os.path.join(TRAIN_PATH,row["Id"]+".jpg")
    wb_table.add_data(row['Id'],
                      wandb.Image(impath),
                      row['Pawpularity'],
                      row['Subject Focus'],
                      row['Eyes'],
                      row['Face'],
                      row['Near'],
                      row['Action'],
                      row['Accessory'],
                      row['Group'],
                      row['Collage'],
                      row['Human'],
                      row['Occlusion'],
                      row['Info'],
                      row['Blur'])
    
wandb.log({'Data Visualization': wb_table})
run.finish()

In [ ]:
frame = IFrame(run.url, width=1280, height=720)
frame

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Pandas Profiling</h2>
    <center><img src = "https://pandas-profiling.github.io/pandas-profiling/docs/assets/logo_header.png" width=200 height = 200></center>
    <p style = "font-family : Lucida Sans Typewriter">
    Pandas profiling is an open source Python module with which we can quickly do an exploratory data analysis with just a few lines of code. Besides, if this is not enough to convince us to use this tool, it also generates interactive reports in web format that can be presented to any person, even if they don’t know programming.
    </p>  
    <a href = "https://pandas-profiling.github.io/pandas-profiling/">Go to offocial website for documentation</a>
</div>

In [ ]:
train_report = ProfileReport(train_df,title="Metadata of Training images")
train_report.to_file("./train_metadata.html")
train_report

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Necessary Functions</h1>

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; ; color:#FF69B4">DataLoader</h2>

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, image_dir, df, data_type):
        self.image_dir = image_dir
        self.df = df
        self.data_type = data_type
        self.filepaths = df["Filepath"].values
        self.labels = df["Pawpularity"].values
        self.meta_features = df[feature_cols].values
        self.data_type = data_type
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image = cv2.imread(self.filepaths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.data_type == "train":
            transforms = A.Compose([
                A.Resize(TRAIN_CONFIG["width"], TRAIN_CONFIG["height"]),
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.2),
                A.Normalize(
                        mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225], 
                        max_pixel_value=255.0, 
                        p=1.0 
                    ),
                ToTensorV2()], p=1.0)
        else:
            transforms = A.Compose([
                A.Resize(TRAIN_CONFIG["width"], TRAIN_CONFIG["height"]),
                A.Normalize(
                        mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225], 
                        max_pixel_value=255.0, 
                        p=1.0 
                    ),
                ToTensorV2()], p=1.0)
            
        image = transforms(image=image)["image"]
        meta_feature = self.meta_features[idx, :]
        label = self.labels[idx]
        
        return image, meta_feature, label

In [ ]:
def create_dataloader(train_idx, val_idx):
    train_ds = PetfinderDataset(TRAIN_PATH, train_df.iloc[train_idx], "train")
    val_ds = PetfinderDataset(TRAIN_PATH, train_df.iloc[val_idx], "valid")
    
    train_loader = DataLoader(train_ds, batch_size=TRAIN_CONFIG["batch_size"],shuffle=True, drop_last=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=TRAIN_CONFIG["batch_size"], shuffle=False, drop_last=True, num_workers=4, pin_memory=True)
    
    return train_loader, val_loader

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; ; color:#FF69B4">Model Function</h2>

In [ ]:
class PetfinderModel(nn.Module):
    def __init__(self, model, pretrained):
        super(PetfinderModel, self).__init__()
        self.base_model = timm.create_model(model, pretrained=pretrained, num_classes=0)
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.LazyLinear(TRAIN_CONFIG["num_classes"])
        
    def forward(self, images, meta_features):
        features = self.base_model(images)
        features = self.dropout(features)
        features = torch.cat([meta_features, features], dim=1)
        output = self.fc(features)
        
        return output

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px;  text-align:center; ; color:#FF69B4">Loss Function</h2>

In [ ]:
def loss_criterion(y_pred, y_true):
    mse_loss = nn.MSELoss()(y_pred.view(-1), y_true.view(-1))
    return torch.sqrt(mse_loss)

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; ; color:#FF69B4">Optimizer Function</h2>

In [ ]:
def model_optimizer():
    return optim.Adam(model.parameters(), lr = TRAIN_CONFIG["learning_rate"])

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px;  text-align:center; ; color:#FF69B4">Callbacks Function</h2>

In [ ]:
def lr_scheduler(optimizer):
    scheduler = LR.CosineAnnealingLR(optimizer, T_max=TRAIN_CONFIG['T_max'], 
                                                   eta_min=TRAIN_CONFIG['min_lr'])
    return scheduler

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px;  text-align:center; ; color:#FF69B4">Training Function</h2>

In [ ]:
def train_one_epoch(model, optimizer, scheduler, data_loader, epoch, device):
    model.train()
    total_loss, total_instances = 0, 0
    y_pred, y_true = [], []
    
    grad_scaler = amp.GradScaler()
    
    print(f"----------Epoch {epoch}/{TRAIN_CONFIG['epochs']}--------------")
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), desc="Training")
    
    for batch_id, (images, meta_features, targets) in pbar:
        images = images.to(device, dtype=torch.float)
        meta_features = meta_features.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        num_instances = targets.size(0)
        
        with amp.autocast():
            preds = model(images, meta_features)
            loss = loss_criterion(preds, targets)
            
        grad_scaler.scale(loss).backward()
        grad_scaler.step(optimizer)
        grad_scaler.update()
        optimizer.zero_grad()
        
        total_loss += (loss.item()*num_instances)
        total_instances += num_instances
        
        y_true.append(targets.view(-1).cpu().detach().numpy())
        y_pred.append(preds.view(-1).cpu().detach().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    
    train_loss = total_loss/total_instances
    train_rmse = mse(y_true, y_pred, squared=False)
    
    gc.collect() 
    
    return train_loss, train_rmse

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px;  text-align:center; ; color:#FF69B4">Validation Function</h2>

In [ ]:
@torch.no_grad()
def val_one_epoch(model, data_loader, epoch, device):
    model.eval()
    total_loss, total_instances = 0, 0
    y_pred, y_true = [], []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), desc="Validation")
    
    for batch_id, (images, meta_features, targets) in pbar:
        images = images.to(device, dtype=torch.float)
        meta_features = meta_features.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        num_instances = targets.size(0)
        
        preds = model(images, meta_features)
        loss = loss_criterion(preds, targets)
        
        total_loss += (loss.item()*num_instances)
        total_instances += num_instances
        
        y_true.append(targets.view(-1).cpu().detach().numpy())
        y_pred.append(preds.view(-1).cpu().detach().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    
    val_loss = total_loss/total_instances
    val_rmse = mse(y_true, y_pred, squared=False)
    
    gc.collect()
    
    return val_loss, val_rmse

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; ; color:#FF69B4">All at One Place</h2>

In [ ]:
def fit_model(model, optimizer, scheduler,train_loader, val_loader, device, epochs, fold):
    if fold == 0:
        wandb.watch(model, log_freq=100)
    
    if TRAIN_CONFIG['device'] == 'cpu':
        print("Using CPU \n")
    else:
        print("Using GPU: {}\n".format(torch.cuda.get_device_name()))
        
    #best_weights = copy.deepcopy(model.state_dict())
    best_val_rmse = np.inf
    history = defaultdict(list)
    model_path = "bestmodel_fold_{:.0f}.bin".format(fold)
    
    start = time.time()
    for epoch in range(1, epochs+1):
        gc.collect()
        
        train_loss, train_rmse = train_one_epoch(model, optimizer, scheduler, train_loader, epoch, device)
        val_loss, val_rmse = val_one_epoch(model, val_loader, epoch, device)
        
        print(f"Train Loss : {train_loss} - Train RMSE : {train_rmse} - Val Loss : {val_loss} - Val RMSE : {val_rmse}")
        
        history["train_loss"].append(train_loss)
        history["train_rmse"].append(train_rmse)
        history["val_loss"].append(val_loss)
        history["val_rmse"].append(val_rmse)
        
        wandb.log({"Train Loss Fold {}".format(fold) : train_loss})
        wandb.log({"Val RMSE Fold {}".format(fold) : val_rmse})
        wandb.log({"Valid Loss Fold {}".format(fold) : val_loss})
        wandb.log({"Valid RMSE Fold {}".format(fold) : val_rmse})
        
        if val_rmse <= best_val_rmse:
            print(f"{b_crma}Validation RMSE Improved from {best_val_rmse} to {val_rmse}, saving model to {model_path} {sr_crma} \n")
            best_val_rmse = val_rmse
            #best_weights = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), model_path)
    
        else:
            print(f"{r_crma}Validation RMSE didn't improve from {best_val_rmse} {sr_crma} \n")
    
    end = time.time()
    time_elapsed = end - start
    print('Training completed in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print(f"{y_crma}Best Val RMSE: {best_val_rmse} {sr_crma}")
    
    #model.load_state_dict(best_weights)
    gc.collect()
    return history

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Training</h1>

In [ ]:
run = wandb.init(
    project = WANDB_CONFIG["project_name"],
    group = WANDB_CONFIG["group_name"],
    job_type = WANDB_CONFIG["job_type_train"],
    anonymous= WANDB_CONFIG["anonymity"]
)

In [ ]:
kfold = StratifiedKFold(n_splits=TRAIN_CONFIG["n_splits"], shuffle=True, random_state=TRAIN_CONFIG["seed"])
history_dict = {}

for fold, (tID, vID) in enumerate(kfold.split(files_ls, pseudo_labels)):
    print(f"Working in fold {fold+1}")
    train_loader, val_loader = create_dataloader(tID, vID)
    
    model = PetfinderModel(TRAIN_CONFIG['model'], pretrained=TRAIN_CONFIG["pretrained"])
    model.to(TRAIN_CONFIG['device'])
    model(rand_img, rand_meta)
    
    optimizer = model_optimizer()
    scheduler = lr_scheduler(optimizer)
    
    history_dict[fold] = fit_model(model, optimizer, scheduler, 
                                   train_loader=train_loader,
                                   val_loader=val_loader,
                                   device=TRAIN_CONFIG["device"],
                                   epochs=TRAIN_CONFIG["epochs"],
                                   fold = fold)
    print("\n\n")

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">History of Training</h1>

In [ ]:
frame = IFrame(run.url, width=1280, height=720)
frame

<center><img src = "https://img.shields.io/badge/Completed-The%20End-brightgreen" width=400 height = 400></center>